The Agricultural model determines the yield and production, as well as the costs of cultivation, for agricultural products.

Yields are estimated using a linear expression:
$$log y_t = \alpha + \gamma g_t + \kappa k_t + \omega w_t + \tau t$$

where $y_t$ are the yields, in crop-specific units, per acre; $g_t$ and $k_t$ are growing-degree days and extreme degree-days, with crop-specific limtis; $w_t$ is the season-cumulative water deficit; and $t$ is the year.  Greek letters denote calibrated coefficients.

Currently, nation-wide coefficients for each crop are determined by a pooled regression of available data.  These are then combined with county-specific for counties with observed data, according to:
$$\beta \sim \mathcal{N}\left(\frac{\frac{\hat\beta}{\hat\sigma^2} + \frac{\bar\beta}{\bar\sigma^2}}{\frac{1}{\hat\sigma^2} + \frac{1}{\bar\sigma^2}}, \frac{1}{\frac{1}{\hat\sigma^2} + \frac{1}{\bar\sigma^2}}\right)$$

where $\hat\beta$ is a given county unpooled coefficient (one of $\alpha$, $\gamma$, $\kappa$, $\omega$, and $\tau$), $\hat\sigma$ is its standard error, and $\bar\beta$ and $\bar\sigma$ are the corresponding national pooled coefficients.

This will be replaced with county-specific parameters from [the hierarchical method](https://www.overleaf.com/read/grjyvqjdgqgc), when these are available.

The list (and order!) of crops is defined in `world.jl`:

In [1]:
crops = ["alfalfa", "otherhay", "Barley", "Barley.Winter", "Maize", "Sorghum", "Soybeans", "Wheat", "Wheat.Winter"]

9-element Array{ASCIIString,1}:
 "alfalfa"      
 "otherhay"     
 "Barley"       
 "Barley.Winter"
 "Maize"        
 "Sorghum"      
 "Soybeans"     
 "Wheat"        
 "Wheat.Winter" 

The key crop-specific definitions are listed in the table below.

In [2]:
using DataFrames
readtable("../prepare/agriculture/cropmapping.csv")[:, [:crop, :unit, :yieldfile, :deficitfile, :eddprefix1, :eddprefix2, :gdd0, :kdd0, :notes]]

,crop,unit,yieldfile,deficitfile,eddprefix1,eddprefix2,gdd0,kdd0,notes
1,sorghum,lb,sorghum_yield_in_lb_per_acre.csv,sorghum__unit_area_deficit_in_meter.csv,Sorghum,Sorghum.2,8,29,From SorghumGDD.CRO - 1
2,hay,lb,hay_yield_in_lb_per_acre.csv,hay__unit_area_deficit_in_meter.csv,alfalfa,otherhay,0,30,Made it up.
3,wheat,bu,wheat_yield_in_bu_per_acre.csv,wheat__unit_area_deficit_in_meter.csv,Wheat,Wheat.Winter,0,26,From WheatGDD.CRO
4,soybeans,bu,soybeans_yield_in_bu_per_acre.csv,soybean__unit_area_deficit_in_meter.csv,Soybeans,NA,5,29,From SoybeanGDD.CRO - 1
5,barley,bu,barley_yield_in_bu_per_acre.csv,barley__unit_area_deficit_in_meter.csv,Barley,Barley.Winter,0,15,From BarleyGDD.CRO
6,corn,bu,corn_yield_in_bu_per_acre.csv,corn__unit_area_deficit_in_meter.csv,Maize,Maize.2,8,29,From Shlenker and Roberts
7,cotton,lb,cotton_yield_in_lb_per_acre.csv,cotton__unit_area_deficit_in_meter.csv,Cotton,NA,12,31,From CottonGDD.CRO cap at 31
8,rice,lb,rice_yield_in_lb_per_acre.csv,rice__unit_area_deficit_in_meter.csv,Rice,Rice.2,8,29,From PaddyRiceGDD.CRO - 1


The yield file and deficit files are available from Naresh.  The EDD suffixes describe the source of season spans, from [Sacks et al. 2010](https://nelson.wisc.edu/sage/data-and-models/crop-calendar-dataset/index.php) and [USDA 1997](http://swat.tamu.edu/media/90113/crops-typicalplanting-harvestingdates-by-states.pdf).  The information for agricultural production comes from quickstats.

In the model, the coefficients on temperature, $\gamma$ and $\kappa$ are not used, and $\tau$ is not used (but all regressions have time centered on 2000).  Instead, irrigated regions are assumed to have $\bar{y}_t = e^\alpha$.  Rainfed areas have yields $\ddot{y}_t = e^{\alpha + \omega (m - p_t)}$, where $m$ is the water requirement of each crop, determined as the maximum water deficit observed for that crop from Naresh's data.  $p_t$ is the season-total precipitation, with seasons currently corresponding to timesteps (typically 6 months).

The water requirements by crop are below, in m.

In [5]:
todata = "../data"
include("../src/Agriculture.jl")
water_requirements

Loading from saved region network...


Dict{ASCIIString,Float64} with 9 entries:
  "otherhay"      => 1.63961100235402
  "Barley"        => 1.18060761343329
  "alfalfa"       => 1.63961100235402
  "Barley.Winter" => 1.18060761343329
  "Soybeans"      => 1.37599595071683
  "Wheat"         => 0.684836198198068
  "Wheat.Winter"  => 0.684836198198068
  "Sorghum"       => 1.1364914374721
  "Maize"         => 1.47596435526564

The total production for a county is
$$q_t = \bar{y}_i \bar{a}_i + \ddot{y}_i \ddot{a}_i$$

where $\bar{a}_i$ is the area used for irrigated production, and $\ddot{a}_i$ is the area used for rainfed irrigation.

According to the definitions above, irrigated regions are fully and exactly irrigated.  Irrigation water demand for each crop is $\bar{a}_i (m - p_t)$

The Agriculture component also defines cultivation costs, below, as USD per acre, from some source.

In [6]:
cultivation_costs

Dict{ASCIIString,Float64} with 9 entries:
  "otherhay"      => 306.0
  "Barley"        => 442.0
  "alfalfa"       => 306.0
  "Barley.Winter" => 442.0
  "Soybeans"      => 221.0
  "Wheat"         => 263.0
  "Wheat.Winter"  => 263.0
  "Sorghum"       => 314.0
  "Maize"         => 554.0

![Known crops](images/knowncrops.png "Known crops")